<center>
## МАТЕМАТИЧЕСКИЙ АППАРАТ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

<img src='../img/Prob_logreg1.png'>

In [139]:
# загружаем необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
plt.rc('font', family='Verdana')

In [140]:
# загружаем данные
data = pd.read_csv("C:/Trees/Bankloan.csv", encoding='cp1251', sep=';')
data.head()

,age,employ,address,debtinc,creddebt,default
0,28,7,2,"17,7","2,990592",0
1,64,34,17,"14,7","5,047392",0
2,40,20,12,"4,8","1,042368",0
3,30,11,3,"34,5","1,75122",0
4,25,2,2,"22,4","0,75936",1


In [141]:
# заменяем запятые на точки и переводим в тип float
for i in ['debtinc', 'creddebt']:
    if i in data.columns:
        data[i]=data[i].str.replace(',', '.').astype('float')

In [142]:
# выводим первые 5 наблюдений
data.head()

,age,employ,address,debtinc,creddebt,default
0,28,7,2,17.7,2.990592,0
1,64,34,17,14.7,5.047392,0
2,40,20,12,4.8,1.042368,0
3,30,11,3,34.5,1.751220,0
4,25,2,2,22.4,0.759360,1


In [143]:
# преобразовываем в тип object
data['default']=data['default'].astype('object')

In [144]:
# выводим информацию о переменных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
age         1500 non-null int64
employ      1500 non-null int64
address     1500 non-null int64
debtinc     1500 non-null float64
creddebt    1500 non-null float64
default     1500 non-null object
dtypes: float64(2), int64(3), object(1)
memory usage: 70.4+ KB


In [145]:
# выполняем дамми-кодирование
print("Исходные переменные:\n", list(data.columns), "\n")
data_dummies = pd.get_dummies(data)
print("Переменные после get_dummies:\n", list(data_dummies.columns))

Исходные переменные:
 ['age', 'employ', 'address', 'debtinc', 'creddebt', 'default'] 

Переменные после get_dummies:
 ['age', 'employ', 'address', 'debtinc', 'creddebt', 'default_0', 'default_1']


In [ ]:
# создаем массив меток и массив признаков

In [146]:
y = data_dummies.loc[:, 'default_1']
data_dummies.drop('default_0', axis=1, inplace=True)
data_dummies.drop('default_1', axis=1, inplace=True)
X = data_dummies.loc[:, 'age':'creddebt']

In [ ]:
# создаем обучающий и тестовый массивы
# признаков и меток

In [147]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [148]:
# применяем RobustScaler
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# импортируем функцию roc_auc_score
from sklearn.metrics import roc_auc_score

# импортируем класс LogisticRegression
from sklearn.linear_model import LogisticRegression

# строим модель логистической регрессии на данных,
# отмасштабированных с помощью RobustScaler
logreg = LogisticRegression().fit(X_train_scaled, y_train)
print("AUC на обучающей выборке: {:.3f}".
      format(roc_auc_score(y_train, logreg.predict_proba(X_train_scaled)[:, 1])))
print("AUC на контрольной выборке: {:.3f}".
      format(roc_auc_score(y_test, logreg.predict_proba(X_test_scaled)[:, 1])))

AUC на обучающей выборке: 0.848
AUC на контрольной выборке: 0.832


In [149]:
# выводим бета-коэффициенты
logreg.coef_

array([[-0.56889341, -2.20105503, -0.01326422,  0.8788113 ,  0.87526835]])

In [150]:
# запишем бета-коэффициенты и названия предикторов
# в отдельные объекты
coef=logreg.coef_
feat_labels = X.columns

In [151]:
# для удобства сопоставим каждому названию 
# предиктора соответствующий бета-коэффициент
for c, feature in zip(coef[0], feat_labels):
    print(feature, c)

age -0.568893408455
employ -2.20105502509
address -0.0132642226832
debtinc 0.878811302173
creddebt 0.875268353245


In [152]:
# вычислим экспоненциальные коэффициенты
# и запишем их в отдельный объект
exp_coef=np.exp(coef)
exp_coef

array([[ 0.56615159,  0.11068632,  0.98682336,  2.40803558,  2.39951913]])

In [153]:
# для удобства сопоставим каждому названию 
# предиктора соответствующий 
# экспоненциальный коэффициент
for c, feature in zip(exp_coef[0], feat_labels):
    print(feature, c)

age 0.566151590752
employ 0.110686319895
address 0.986823359455
debtinc 2.40803557781
creddebt 2.39951912632


In [154]:
# вычислим свободный член (константу)
intercept=logreg.intercept_
intercept

array([-0.89234106])

In [115]:
predvalue = logreg.predict(X_train_scaled)
predvalue

array([1, 0, 0, ..., 1, 0, 0], dtype=uint8)

In [116]:
predvalue=pd.DataFrame(predvalue, index=X_train.index, columns=['Predicted_class'])
predvalue.head()

,Predicted_class
485,1
527,0
199,0
889,0
844,1


In [117]:
prob = logreg.predict_proba(X_train_scaled)
prob

array([[ 0.49565917,  0.50434083],
       [ 0.77963554,  0.22036446],
       [ 0.64022823,  0.35977177],
       ..., 
       [ 0.23357302,  0.76642698],
       [ 0.95415838,  0.04584162],
       [ 0.97300933,  0.02699067]])

In [118]:
probabilities=pd.DataFrame(prob, index=X_train.index, columns=['Prob0', 'Prob1'])
probabilities.head()

,Prob0,Prob1
485,0.495659,0.504341
527,0.779636,0.220364
199,0.640228,0.359772
889,0.798713,0.201287
844,0.429674,0.570326


In [119]:
X_tr=pd.DataFrame(X_train_scaled, index=X_train.index, columns=feat_labels)
X_tr.head()

,age,employ,address,debtinc,creddebt
485,-0.555556,-0.3,-0.571429,-0.022989,-0.061742
527,-0.166667,0.0,0.000000,-0.287356,-0.243909
199,0.111111,0.0,-0.142857,0.091954,0.338743
889,2.222222,-0.2,1.142857,0.160920,0.241992
844,-0.611111,-0.4,-0.571429,0.126437,-0.195654


In [120]:
result=pd.concat([X_tr, y_train, predvalue, probabilities], axis=1)
result.head()

,age,employ,address,debtinc,creddebt,default_1,Predicted_class,Prob0,Prob1
485,-0.555556,-0.3,-0.571429,-0.022989,-0.061742,0,1,0.495659,0.504341
527,-0.166667,0.0,0.000000,-0.287356,-0.243909,0,0,0.779636,0.220364
199,0.111111,0.0,-0.142857,0.091954,0.338743,0,0,0.640228,0.359772
889,2.222222,-0.2,1.142857,0.160920,0.241992,0,0,0.798713,0.201287
844,-0.611111,-0.4,-0.571429,0.126437,-0.195654,0,1,0.429674,0.570326


Давайте вручную вычислим вероятность дефолта для наблюдения 485. 

Подставляем в формулу

$$\Large \frac{1}{1+e^{-(b_0+b_1x_i^{(1)}+b_2x_i^{(2)}+...+b_kx_i^{(k)})}}$$ 

коэффициенты и значения предикторов для данного наблюдения.

<img src='../img/Prob_logreg.png'>


Найденное значение практически совпадает со значением вероятности дефолта, вычисленным автоматически (0.504341). Небольшое различие обусловлено ошибкой округления.